In [9]:
import numpy as np
import cv2

image = cv2.imread("image.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32)
h, w = image.shape
image = image[:h - h % 2, :w - w % 2]  #  чётный размеров

np.savetxt("original.txt", image, fmt="%.0f")
original_size = image.nbytes

In [10]:
def haar2d(img):
    h, w = img.shape
    temp = np.zeros_like(img)

    # Преобразование по строкам
    for i in range(h):
        avg = (img[i, 0::2] + img[i, 1::2]) / 2
        diff = (img[i, 0::2] - img[i, 1::2]) / 2
        temp[i, :w//2], temp[i, w//2:] = avg, diff

    result = np.zeros_like(temp)
    # Преобразование по столбцам
    for j in range(w):
        avg = (temp[0::2, j] + temp[1::2, j]) / 2
        diff = (temp[0::2, j] - temp[1::2, j]) / 2
        result[:h//2, j], result[h//2:, j] = avg, diff

    LL = result[:h//2, :w//2]
    LH = result[:h//2, w//2:]
    HL = result[h//2:, :w//2]
    HH = result[h//2:, w//2:]
    return LL, LH, HL, HH

LL, LH, HL, HH = haar2d(image)

In [11]:

#Квантование
def quantize(block, levels=4):
    min_v, max_v = np.min(block), np.max(block)
    if max_v == min_v:
        return np.zeros_like(block, dtype=int)
    step = (max_v - min_v) / (levels - 1)
    return np.round((block - min_v) / step).astype(int)

LH_q = quantize(LH)
HL_q = quantize(HL)
HH_q = quantize(HH)


In [12]:
#Кодирование длин серий
def rle_encode(array):
    flat = array.flatten()
    encoded = []
    prev = flat[0]
    count = 1
    for val in flat[1:]:
        if val == prev:
            count += 1
        else:
            encoded.append((prev, count))
            prev, count = val, 1
    encoded.append((prev, count))
    return encoded

LH_rle = rle_encode(LH_q)
HL_rle = rle_encode(HL_q)
HH_rle = rle_encode(HH_q)



In [13]:
#Сохранение в файл
with open("wavelet_data.txt", "w") as f:
    np.savetxt(f, LL, fmt="%.2f")
    f.write("\nLH:\n")
    for v, c in LH_rle:
        f.write(f"{v} {c}\n")
    f.write("\nHL:\n")
    for v, c in HL_rle:
        f.write(f"{v} {c}\n")
    f.write("\nHH:\n")
    for v, c in HH_rle:
        f.write(f"{v} {c}\n")

In [14]:
with open("wavelet_data.txt", "r") as f:
    compressed_size = len(f.read().encode("utf-8"))

ratio = original_size / compressed_size
saving = (1 - compressed_size / original_size) * 100

print(f"Исходный размер: {original_size} байт")
print(f"После сжатия: {compressed_size} байт")
print(f"Коэффициент сжатия: {ratio:.2f}x")
print(f"Экономия: {saving:.1f}%")

Исходный размер: 2885120 байт
После сжатия: 1196502 байт
Коэффициент сжатия: 2.41x
Экономия: 58.5%
